<a href="https://colab.research.google.com/github/easyhardhoon/machine_learning/blob/master/ML5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# loss 함수로 가장 많이 쓰이는 두가지.
# 1. 오차제곱합
import numpy as np
def sum_squares_error(y,t):
  return 0.5 * np.sum((y-t)**2) # y는 신경망의 출력 값 , t는 원핫인코딩 정답 레이블

In [7]:
# 2. 교차 엔트로피 오차
def cross_entropy_error(y,t):
  delta = 1e-7
  return -np.sum(t*np.log(y+delta)) # delta : 자연로그 특성상 0을 입력시 -무한대가 나오지 않도록. 아주 작은 값인 delta를 배치
# 여기서 t*np.log__ 는 손실함수이고,  np.sum(t*np.log__) 는 손실함수의 합, 즉 loss임
# loss가 손실함수가 아니고 손실함수의 합임을 주의하자

In [8]:
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))
y = [0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))
# 오차 값이 더 작은 첫번째 추정이 정답일 가능성이 더 높다고 판단하게 됌. 머신러닝의 원리

0.510825457099338
2.302584092994546


In [9]:
# 훈련데이터가 100개이다 --> 100개의 손실함수를 구하고 loss를 구해야지
# 훈련데이터 100개 학습이 한 epoch.(한 사이클)  ---> model.fit(____epochs =1000  ) 이게 실제 학습 코드인데, 이는 이 사이클을 1000번 돌린다는 뜻
# 위 식을 데이터 N개에 대한 식으로 확장할 수 있고 이 전체를 한번에 묶어 평균 손실함수를 구할 수 있고 이를 토대로 하나의 사이클의 loss를 구할 수 있다
# 여기서 구한 하나의 loss를 1000번 반복하며 loss가 작아지는 쪽으로 학습하여 최적의 매개변수(가중치, bias 등)를 찾는 알고리즘이 신경망 학습 알고리즘
# 근데 훈련데이터가 6만개..수천만개가 되면 현실적이지 않음. 일부만 골라 학습을 수행. 이를 미니배치 학습이라고 함

In [10]:
from mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)
print(x_train.shape)
print(t_train.shape)
#x_train의 shape가 60000, 784 라는 뜻  ---> 훈련 데이터가 6만개이다. 하나의 데이터는 28*28 이미지 데이터이다.
#t_train의 shape가 60000, 10 이라는 뜻 ---> 훈련 데이터가 6만개이다. 정답 레이블은 0~9 결과의 10줄짜리 데이터이다.

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!
(60000, 784)
(60000, 10)


In [11]:
x_train.shape[0]

60000

In [12]:
#미니배치 학습 코드
train_size = x_train.shape[0] #6만
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) #그냥 0이상 6만 미만의 수 중 10개 선택임해서 배열 만들어줌
print(batch_mask)
x_batch = x_train[batch_mask] # 10개 랜덤 인덱스. 여기있는 값만 추후에 학습데이터로 삼겠다!
t_batch = t_train[batch_mask]

[44036 46421 35507 58666 57205 12244  8971 52486 17761 24122]


In [13]:
def cross_entropy_error(y,t):
  if y.ndim == 1:
    t = t.reshape(1,t.size)
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(t*np.log(y+1e-7)) / batch_size
  

In [14]:
def cross_entropy_error(y,t):
  if y.ndim == 1: # 데이터가 1차원일때. 다차원일떄를 고려한 코드, 즉 여러개가 batch로 한꺼번에 들어올때를 대비한 코드
    t = t.reshape(1,t.size)
    y = y.reshape(1,y.size)
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
  # 정답 레이블이 원핫인코딩이 안되어있을때
  # 원핫인코딩일때 t가 0인 원소는 교차 엔트로피 오차도 0이었으므로 전체 로직은 변하지 않는다가 핵심
  # 그러므로 여기서도 정답에 해당하는 신경망의 출력만으로 교차 엔트로피 오차를 구할수 있다가 핵심
  # 그래서 ex) y[0,2], y[1,7], y[3,9] .. 의 값들(각 데이터 0 ,1 ,2 ...에서 정답에 해당하는 인덱스의 값 2, 7, 9 ....) 만 더해주면 됌. 
  # 최종적으로 구하는 것은 하나의 loss의 값이기 때문에 평균손실함수의 개념을 써서 batch_size로 나눔
  # 이렇게 해서 구해진 하나의 loss. 

In [15]:
# 교차 엔트로피 오차는 "정답일때의" 출력만으로만 이루어져있다

In [16]:
# 신경망을 학습할떄 정확도가 아닌 loss를 설정하는 이유
# 가중치 매개변수의 미분 값이 양수/ 음수임에 따라 loss를 설정하는 원리 때문. 정확도를 지표로 하게 되면 매개변수의 미분이 대부분 0이 나옴

**정확도를 지표로 하게 되면 매개변수의 미분이 대부분 0이 나오는 이유**

신경망에서 활성화함수로 계단함수를 안쓰는 이유와 마찬가지임. 매개변수의 미묘한 변화에 바로 반응을 보여야 함 . 
신경망 학습의 원리는 매개변수의 끊임없는 수정임. 작은 변화에도 즉각적으로 연속적으로 변해야함. 
이러한 이유로 loss를 쓰는 것이며 활성화함수로 비선형 & 매끈한 함수를 쓰는것임 ( 미분은 함수의 기울기와 관련이 있다)
+ 선형함수이면 기울기가 일정하기 때문에 마찬가지로 신경망에서 사용안함

In [17]:
def numercial_diff(f,x):
  h = 1e-50  #1e-50은 너무 작음. 32비트 float 머신에서는 반올림오차로 인해 그냥 0.0으로 인식해버림. 10*-4 가 좋은 결과를 얻는다고 알려져있음
  return (f(x+h)- f(x))/h
# 두번째 문제. 결국은 h를 무한히 0으로 좁히는것이 불가능하기 때문에 진정한 미분이라고 할 수 없음. m1과 m2는 결국 같아질수 없다. 
# m1 : f(x)가 x에서의 접선의 기울기
# m2 : f(x) , f(x+h) 사이의 기울기. h는 최대한 0으로 근사

미분 : 한순간의 변화량.
x의 작은 변화가 f(x)를 얼마나 변화시켰느냐

In [18]:
# h를 무한히 0으로 근사할수는 없다. 오차를 최대한  줄이기 위해서 중심 차분 개념 사용
def numercial_diff(f,x):
  h =1e-4 #참고로 e는 자연로그가 아니고 10임
  return (f(x+h) - f(x-h)) / (2*h) #  x에 대한 f(x)의 변화량. 즉 함수의 기울기.  ex) f(x)의 x=3에서의 접선의 기울기.
  # 실제로 수식에 대입해 return 한값과 수학에서 배운 미분값을 비교해보면 오차가 매우 작음

In [20]:
def function_2(x):
  return x[0]**2 + x[1]**2
  #return np.sum(x**2) 도 가능
  # 기본 형식이 f(x0,x1). 변수가 2개 들어감. 편미분의 개념
  # f(X0,X1) 이런 함수를 그래프로 그리면 3차원으로 그려짐

x0 = 3, x1 = 4 일때 x0에 대한 편미분 &f/&x0 을 구하라

In [21]:
def function_tmp(x0):
  return x0*x0 + 4.0**2.0
numercial_diff(function_tmp,3.0) 
#이처럼 편미분도 변수가 하나인 미분과 마찬가지로 특정 장소의 기울기를 구한다.
#다만 여러 변수 중 목표 변수 하나에 초점을 맞추고 다른 변수는 값을 고정함. 그러기 위해 새로운 함수를 정의하고 
# 이 함수에 대해 그동안 사용한 수치 미분 함수를 적용하여 편미분을 구한 것임

6.00000000000378